In [1]:
# import all modules
# Third-party
import numpy as np
import torch

# Local files
import utilities
import train_NN as train
import neural_network as net

# Choose a device to run on - either CPU or GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
# Choose a ground truth PDE to analyze
a = 4 # PDE parameter

# True PDE - Network will satify this PDE at the end of training
def pde_fn(x, a):
    # PDE: u_xx = -(pi*a)^2.sin(pi*a*x)
    u_xx = -(np.pi * a)**2 * np.sin(np.pi * a * x)
    return u_xx

# True solution - Network will try to learn this
def u(x, a):
    return np.sin(np.pi * a * x)

In [3]:
# Create a dataset by sampling points from the PDE's domain
    #1. For the boundaries - subscript u
    #2. Inside the domain = residual - subscript r
# Here, 1D domain, so boundary is just two points! 
bc1_coords = np.array([[0.0], 
                       [0.0]]) 

bc2_coords = np.array([[1.0], 
                       [1.0]])

dom_coords = np.array([[0.0],
                       [1.0]])

no_of_data_samples  = 100
X_bc1 = dom_coords[0, 0] * np.ones((no_of_data_samples // 2, 1))
X_bc2 = dom_coords[1, 0] * np.ones((no_of_data_samples // 2, 1))

X_u = np.vstack([X_bc1, X_bc2]) # data for BC
Y_u = u(X_u, a) # y data for BC

X_r = np.linspace(dom_coords[0, 0],
                  dom_coords[1, 0], no_of_data_samples)[:, None] # data for residual
Y_r = pde_fn(X_r, a) # y data for residual

In [4]:
# Define model and a random seed
seed = 1
max_iterations = 10
model = net.PINN(no_of_neurons=50, no_of_h_layers=2)
model.to(device)

# Train the model -> default SGD with full batch and exponential weight decay
save_data_location = './data_ntk/'
details = train.train_nn_model(model, train_data=(X_u, Y_u, X_r, Y_r),
                            no_iterations=max_iterations, device=device,
                            save_data_location=save_data_location,
                            save_data_frequency=1)
details.keys()

dict_keys(['Loss', 'L_b', 'L_r'])

In [8]:
# Calculate the eigenvalues
import pickle
# load the jacobian matrices from hard disk
J_u, J_r = pickle.load(open("{}/Ju_Jr_{}.p".format(save_data_location,0),
                            'rb'))
eigs = utilities.calculate_eigenvalues_from_j(J_u, J_r)

In [10]:
eigs

(100,)